In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

In [3]:
# load cleaned data
df = pd.read_csv('cleaned_nans_removed.csv')
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_brand_bool,prop_location_score1,prop_log_historical_price,...,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool
0,81578,1351728509,5,219,219,93974,4,1,5.17,5.72,...,17012,1,15,2,0,1,1,1,0,0
1,81578,1351728509,5,219,219,133689,4,1,5.19,5.75,...,17012,1,15,2,0,1,1,1,0,0
2,81578,1351728509,5,219,219,119527,4,1,5.18,5.82,...,17012,1,15,2,0,1,1,1,0,0
3,81578,1351728509,5,219,219,95095,5,1,4.93,6.21,...,17012,1,15,2,0,1,1,1,0,0
4,81578,1351728509,5,219,219,4080,4,1,5.18,5.84,...,17012,1,15,2,0,1,1,1,0,0


In [11]:
pos = 32
positions = list(df[df['srch_id']== 81578]['position'])
(len(positions) - pos) / len(positions)

0.030303030303030304

In [14]:
def relevance_score_click_book(qd):
    if qd['booking_bool'] == 1:
        return 2
    if qd['click_bool'] == 1:
        return 1
    return 0

def relevance_score_position(qd):
    query_id = qd['srch_id']
    pos = qd['position']
    positions = list(df[df['srch_id']== query_id]['position'])
    score = (len(positions) - pos) / len(positions)
    return score

df["relevance"] = df.apply(relevance_score_position,axis=1)

KeyboardInterrupt: 

In [6]:
cutoff_id = df["srch_id"].quantile(0.94) # 90/10 split
x_train = df.loc[df.srch_id< cutoff_id].drop(['click_bool', 'booking_bool', 'relevance'],axis=1)
x_eval = df.loc[df.srch_id>= cutoff_id].drop(['click_bool', 'booking_bool', 'relevance'],axis=1)
y_train = df.loc[df.srch_id< cutoff_id]["relevance"]
y_eval = df.loc[df.srch_id>= cutoff_id]["relevance"]

In [7]:
log_res = LogisticRegression()
log_res.fit(x_train, y_train)

LogisticRegression()

In [8]:
predictions = log_res.predict(x_eval)

In [9]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
score = log_res.score(x_eval, y_eval)
print(score)

0.9552661210359141


In [14]:
import torch
probs = log_res.predict_proba(x_eval)
torch.tensor(probs).shape

In [ ]:
probabilities = log_res.predict_proba(x_eval)[:, 1]  # Probabilities of class 1 (relevant)

# Combine probabilities with original data indices for ranking
# Example: Create a list of (index, probability) tuples
items_with_probabilities = list(zip(range(len(X)), probabilities))

# Sort items based on predicted probabilities (descending order)
items_with_probabilities_sorted = sorted(items_with_probabilities, key=lambda x: x[1], reverse=True)

# Display the ranked items
print("Ranking based on predicted probabilities:")
for rank, (index, probability) in enumerate(items_with_probabilities_sorted, start=1):
    print(f"Rank {rank}: Item {index} - Probability: {probability:.4f}")